# Group Stage Lineups Generator

In [1]:
import pandas as pd
from scipy.optimize import linprog
from enum import Enum
from enums.countries import Countries as Teams
from pulp import *

* Load data created by generate_players_database.ipynb

In [2]:
data = pd.read_csv('output/all_rounds_players_predicted.csv', encoding='utf-8-sig')

In [3]:
data.head()

,name,price,points,position,team,Apps,Mins,Goals,Assists,SpG,...,gfpg,gapg,opponent_gfpg,opponent_gapg,pr_diff,pred_GF,pred_GA,pred_score,round,pred_points
0,הוגו לוריס,10,0,GK,צרפת,9,810,0,0,0.0,...,2.00,0.72,3.00,1.05,0.197,1.749,1.649,"(1, 1)",1,2.0
1,אונאי סימון,9,0,GK,ספרד,5,450,0,0,0.0,...,2.48,0.60,1.84,1.08,0.170,2.001,1.099,"(2, 1)",1,2.0
2,דויד דה חאה,6,0,GK,ספרד,4,360,0,0,0.0,...,2.48,0.60,1.84,1.08,0.170,2.001,1.099,"(2, 1)",1,0.0
3,טיבו קורטואה,9,0,GK,בלגיה,4,360,0,0,0.0,...,3.20,0.36,2.00,0.96,0.552,3.244,0.868,"(3, 0)",1,6.0
4,סיימון מיניולה,4,0,GK,בלגיה,4,395,0,0,0.0,...,3.20,0.36,2.00,0.96,0.552,3.244,0.868,"(3, 0)",1,0.0


## User interface
1. MAX_SAME_TEAM - maximal number of players from the same team allowed.
2. BUDGET - budget allowed in groups stage.
3. NUM_SUBS - number of allowed subs between each match.
4. NUM_ROUNDS - number of rounds from the beginning of the groups stage to optimize lineup on.
5. FORMATION - chosen formation to use during groups stage
6. FILTER_PLAYERS - filter players by set of attributes.

* Filters

* ONLY_STARTERS_FILTER - get only starters
* MINIMAL_PRICE_FILTER - set minimal price for players
* GK_MIN_RATING_FILTER - get only goalkeepers with (whoscored.com) rating higher then this
* D_MIN_RATING_FILTER - get only defenders with (whoscored.com) rating higher then this
* M_MIN_RATING_FILTER - get only midfielders with (whoscored.com) rating higher then this
* FW_MIN_RATING_FILTER - get only forwards with (whoscored.com) rating higher then this
* D_MIN_GNA_FILTER - get only defenders involved in higher percentage of goals and assists then this
* M_MIN_GNA_FILTER - get only midfielders involved in higher percentage of goals and assists then this
* FW_MIN_GNA_FILTER - get only forwards involved in higher percentage of goals and assists then this

In [4]:
##############################################################################################################
MAX_SAME_TEAM = 2              # {2,3,..,8}
BUDGET = 100.0                 # {100.0, 105.0, 110.0, 115.0}
NUM_SUBS = 3                   # {3,4,...,11}
NUM_ROUNDS = 3                 # {2,3}
FORMATION = {'GK': 1, 'D': 5, 'M': 3, 'FW': 2} # Note: [5,3,2],[4,3,3] are usualy the best formations (by pts)
##############################################################################################################
FILTER_PLAYERS = False
## the following configs are enabled only if FILTER_PLAYERS=True
ONLY_STARTERS_FILTER = True     # {True, False}
MINIMAL_PRICE_FILTER = 4        # {3,4,5,...,10}
# rating as calculated by whoscored.com
GK_MIN_RATING_FILTER = 6.3      # [0.0, 10.0], Note: GK mean rating = ~6.6
D_MIN_RATING_FILTER = 7.0       # [0.0, 10.0], Note: D mean rating = ~7.0
M_MIN_RATING_FILTER = 7.2       # [0.0, 10.0], Note: M mean rating = ~7.2
FW_MIN_RATING_FILTER = 7.4      # [0.0, 10.0], Note: FW mean rating = ~7.4
# GNA in the percentage of goals and assists player is involved in out of the team's total goals and assists
D_MIN_GNA_FILTER = 0.01         # [0.0, 1.0]
M_MIN_GNA_FILTER = 0.01         # [0.0, 1.0]
FW_MIN_GNA_FILTER = 0.1         # [0.0, 1.0]
##############################################################################################################

## Filter players pool (if FILTER_PLAYERS=True)

In [5]:
columns = ['name', 'team', 'position', 'price', 'pred_points', 'round']

if FILTER_PLAYERS:

    filter_columns = ['name', 'position', 'price', 'pred_points', 'team', 'pred_score', 'opponent', 'Rating', 'Mins_percentage', 'GnA_percentage', 'is_starter', 'round']
    filtered_data = data[filter_columns].copy()

    if ONLY_STARTERS_FILTER:
        filtered_data = filtered_data[filtered_data['is_starter'] == 1]
    filtered_data.drop(columns=['is_starter'], inplace=True)
    
    filtered_data = filtered_data[filtered_data['price'] >= MINIMAL_PRICE_FILTER]
    
    goalkeepers_pool = filtered_data[filtered_data['position'] == 'GK']
    defenders_pool = filtered_data[filtered_data['position'] == 'D']
    midfielders_pool = filtered_data[filtered_data['position'] == 'M']
    forwards_pool = filtered_data[filtered_data['position'] == 'FW']

    goalkeepers_pool = goalkeepers_pool[goalkeepers_pool['Rating'] >= GK_MIN_RATING_FILTER]
    goalkeepers_pool.sort_values('pred_points', ascending=False).head()

    defenders_pool = defenders_pool[defenders_pool['Rating'] >= D_MIN_RATING_FILTER]
    defenders_pool = defenders_pool[defenders_pool['GnA_percentage'] >= D_MIN_GNA_FILTER]
    defenders_pool.sort_values('pred_points', ascending=False).head()

    midfielders_pool = midfielders_pool[midfielders_pool['Rating'] >= M_MIN_RATING_FILTER]
    midfielders_pool = midfielders_pool[midfielders_pool['GnA_percentage'] >= M_MIN_GNA_FILTER]
    midfielders_pool.sort_values('pred_points', ascending=False).head()

    forwards_pool = forwards_pool[forwards_pool['Rating'] >= FW_MIN_RATING_FILTER]
    forwards_pool = forwards_pool[forwards_pool['GnA_percentage'] >= FW_MIN_GNA_FILTER]
    forwards_pool.sort_values('pred_points', ascending=False).head()

    filtered_data = pd.concat([goalkeepers_pool, defenders_pool, midfielders_pool, forwards_pool])
    filtered_data

    players_attributes = filtered_data[columns].copy()
    
else:
    players_attributes = data[columns].copy()

## Transform database (players attributes) to be LP ready

In [6]:
positions = list(FORMATION.keys())
for position in positions:
    players_attributes[f'is_{position}'] = players_attributes['position'] == position
players_attributes.drop('position', axis='columns', inplace=True)

for team in Teams:
    players_attributes[f'is_{team.name}'] = players_attributes['team'] == team.value
players_attributes.drop('team', axis='columns', inplace=True)

In [7]:
%%capture
rounds_df_list = []
for i in range(NUM_ROUNDS):
    round_df = players_attributes[players_attributes['round'] == (i+1)]
    round_df.drop('round', axis='columns', inplace=True)
#     round_df['in'] = True
    round_df.set_index('name', inplace=True)
    for j in range(len(round_df.columns)):
        round_df.rename({(round_df.columns)[j]: f"r{i+1}_" + (round_df.columns)[j]}, axis='columns', inplace=True)
    rounds_df_list.append(round_df)

In [8]:
players_merged = pd.concat(rounds_df_list, axis=1)
players_dup = pd.concat([players_merged]*(2**NUM_ROUNDS))
players_dup_sorted = players_dup.sort_index().reset_index()

In [9]:
in_cols = list(f'r{i+1}_in' for i in range(NUM_ROUNDS))

def add_ins(row):
    index = row.name
    index_mod = index % (2**NUM_ROUNDS)
    bits = f'{index_mod:012b}' # 12 is hard-coded as number of maximum rounds
    bits = bits[-NUM_ROUNDS:]
    return pd.Series([int(i) for i in list(bits)], index=in_cols)

players_dup_sorted[in_cols] = players_dup_sorted.apply(lambda row : add_ins(row), axis=1);

In [10]:
rounds_dfs_list = []
for i in range(NUM_ROUNDS):
    round_df = players_dup_sorted.filter(like=f'r{i+1}_')
    round_multiply_df = round_df.apply(lambda row: row * row[f'r{i+1}_in'], axis=1)
    rounds_dfs_list.append(round_multiply_df)

players_selections = pd.concat(rounds_dfs_list, axis=1)

In [11]:
players_selections['name'] = players_dup_sorted['name']
cols_to_sum = list(f'r{i+1}_pred_points' for i in range(NUM_ROUNDS))
players_selections['total_pred_points'] = players_selections[cols_to_sum].sum(axis=1)
for i in range(NUM_ROUNDS-1):
    players_selections[f'subs_r{i+1}{i+2}'] = players_selections[f'r{i+1}_in'] ^  players_selections[f'r{i+2}_in']

## Solve using descrete Linear Programing (LP)
*with given constraints and limits

In [12]:
def get_multi_rounds_optimal_lineup(players_df, num_rounds):
    
    same_player_cond = True
    cols = list(players_df.columns.values)
    cols.remove('name')
    cnt = 0
    
    while same_player_cond:
        players_df = players_df.copy()
        # set problem
        prob = LpProblem(f"Oplimal_Lineup_Problem_{cnt}",LpMaximize)
        players = list(players_df.index)   

        # set variables
        players_vars = LpVariable.dicts("Players", players, lowBound=0, upBound=1, cat='Integer')

        # set weights and add conditions
        ## player predicted points & set optimization target
        points = dict(zip(players,players_df['total_pred_points']))
        prob += lpSum([points[i]*players_vars[i] for i in players])

        for i in range(num_rounds):
            ## player price & budget constraint
            prices = dict(zip(players,players_df[f'r{i+1}_price']))
            prob += lpSum([prices[f] * players_vars[f] for f in players]) <= BUDGET
            ## player position & choosen formation constraint
            positions_dicts_dict = {}
            for position in positions:
                positions_dicts_dict[position] = dict(zip(players,players_df[f'r{i+1}_is_{position}']))
                prob += lpSum([(positions_dicts_dict[position])[f] * players_vars[f] for f in players]) == FORMATION[position]
            # player team & maximun number of players from same team constraint
            teams_dicts_dict = {}
            for team in Teams:
                teams_dicts_dict[team] = dict(zip(players,players_df[f'r{i+1}_is_{team.name}']))
                prob += lpSum([(teams_dicts_dict[team])[f] * players_vars[f] for f in players]) <= MAX_SAME_TEAM
            ## subs constraint
            if i == (num_rounds - 1):
                break
            subs = dict(zip(players,players_df[f'subs_r{i+1}{i+2}']))
            prob += lpSum([subs[f] * players_vars[f] for f in players]) <= (2 * num_rounds)

        # solve ploblem
        prob.solve()
        print("Status:", LpStatus[prob.status])

        # get optimized lineup
        players_indexes = []
        for v in prob.variables():
            if v.varValue>0:
#                 print(f'name: {v.name[8:]}, amount: {v.varValue}')
                players_indexes.append((v.name[8:]).replace('_', ' '))

        lineup = players_df.iloc[players_indexes]
        
        # if player is selected to be used in more than 1 combination of rounds - remove the lower combination to avoid losing sub.
        # example: is player_X is selected to be used in rounds 1,2,3 and in rounds 1,3 - by removing the 1,3 selection, 2 subs are avoided.
        if lineup['name'].value_counts().max() == 1:
            same_player_cond = False
        else:
            dup_player_idx = lineup[lineup['name'] == lineup['name'].value_counts().idxmax()].sort_values('total_pred_points').head(1).index
            players_df.loc[dup_player_idx,cols] = 0
            cnt += 1
            
    return lineup

In [13]:
%%capture
lineup = get_multi_rounds_optimal_lineup(players_selections, num_rounds=NUM_ROUNDS);

## Print optimal lineup
* name - name of player.
* total_prdicted_points - sum of predicted points to contribute by the player.
* r#_in - is the player in lineup in round #.

In [14]:
total_points = lineup['total_pred_points'].sum()
print(f'Total predicted points in {NUM_ROUNDS} round(s): {total_points:.2f}')
for i in range(NUM_ROUNDS):
    round_used_budget = lineup[f'r{i+1}_price'].sum()
    print(f'Round {i+1} used budget: {round_used_budget}M$')
cols = ['name', 'total_pred_points'] + in_cols
lineup[cols]

Total predicted points in 3 round(s): 218.74
Round 1 used budget: 100M$
Round 2 used budget: 100M$
Round 3 used budget: 100M$


,name,total_pred_points,r1_in,r2_in,r3_in
1025,ג'ייסון דנאייר,7.087,0,0,1
1225,דוייה צ'אלטה-צאר,6.000,0,0,1
1671,ולדימיר דארידה,14.614,1,1,1
1718,ז'ואאו קאנסלו,11.930,1,1,0
2290,לודוויג אוגוסטינסון,7.601,0,1,0
2511,מייסון מאונט,19.818,1,1,1
2615,ממפיס דפאיי,21.123,1,1,1
370,אלכסנדר קראבייב,7.751,0,1,0
3796,רובן דיאס,8.354,1,0,0
3895,רומלו לוקאקו,21.577,1,1,1
